In [ ]:
# Important libraries to use for this project.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

# Introduction
Have you ever wondered from where the sigmoidal function have risen?! Hopefully this notebook will help you in understanding the sigmoidal function and the necessity of it for classification tasks. In order to understand what I am going to explain, you need a bit of knowledge in linear algebra and probability. Don't forget linear algebra exist for optimization and visualization of higher dimensional space, hence, my implementation will use numpy array and hopefully you willn't see too many "for loops" in this notebook ;). A small note anything with an underline indicate that we are dealing with a vector, and $\underline{w}$ will contain b(i.e bias term).
## Prerequisite:
    * Linear Algebra
    * Applied probabilistic models

# Logistic Regression (LR)
LR is one of the discriminative models that is constructed by directly modelling the conditional distribution of the target class(i.e a discriminator not a generative model). This project will proceed in discussing the LR by going through generalized linear model (GLM), in which this will help in reaching to the famous formula for LR. Also, LR deals with binary target and this is indicative that the conditional distribution can be modelled by the Bernoulli distribution in which it is fully described by $\phi$($\phi$ is the mean of the Bernouli distribution), and this distribution is as follows:-
\begin{align}
&Bernoulli(t|x;\phi) = \phi^{t}(1-\phi)^{1-t}, t\in \{0, 1\}; and\ we\ are\ going\ to\ show\ the\ relationship\ between\ \phi\ and\ x\
\end{align}

GLM's represent a generalization for the ordinarily least square (OLS) models, in which the GLM's aren't restricted to the relationship of **linearity** between the mean value of the conditional distribution of the target value and the parameters. And that was explicitly stated by the OLS construction in which the OLS models are built on the assumption of noise that is normally distributed. The GLM's allows generalization for the distribution of the noise which leads into allowing the conditional distribution of the target to have a non-linear relationship with the parameters of the model. This relationship is represented by the following:-  $E[\underline{t}|\underline{x};\underline{w}] = t = f(\underline{w}^T \underline{x})$, where f(.) is a non-linear function. And f(.) is known as the canonical response function, and the inverse of this function is known as the canonical link function. Which essentially act as a link between the linear combination of the parameters and input features with the response of the model(i.e t), and in the LR this link would be the Logit function ( $\eta = f^{-1}(E[\underline{t}|\underline{x};\underline{w}])$).

GLM's involve mainly the transformation of the conditional distribution of the target to the exponential family which are described with the following formula:- $p(x|\eta) = h(x) g(\eta) exp(\eta^Tu(x))$. Where is u(x) is knows as sufficient statistics and the reason for this name will be shown shortly, $g(\eta)$ represent the normalization term that ensure that the exponential distribution is a proper PDF. And its formula is as follows $g(\eta)\int_{-\infty}^{\infty}h(x)exp(\eta^Tu(x))dx=1$. And the $\eta$ parameter is the natural parameter that fully describe the exponential distribution.
#### Bernoulli transformation to exponential
\begin{align}
\begin{split}
&p(c=1|x;\phi) = \phi,\ p(c=0|x;\phi)=1-\phi\\
&p(c|x;\phi) = \phi^{c}(1-\phi)^{1-c}; c \in \{0, 1\}\\
&p(c|x;\phi) = exp\Big( log\big(\phi^{c}(1-\phi)^{1-c}\big)\Big) =\ exp\Big( c\ log(\phi) + (1-c)log(1-\phi)\Big)=\ (1-\phi)exp(\ c\ log(\frac{\phi}{1-\phi}))\\
&u(x) = c,\ \eta^T=\eta=log(\frac{\phi}{1-\phi}),\\
& For\ sigmoidal\ function\ :\ exp(\eta)*(1-\phi)=\phi\  ->\ \phi = \frac{exp(\eta)}{1+\exp(\eta))} = \frac{1}{1+exp(-\eta)} = \sigma(\eta)\\
&\therefore p(c|x;\eta) = \sigma(\eta)^c\ (1-exp(\eta))^{1-c}
\end{split}
\end{align}
The canonical link response in this case the Logit in which it is expressed by $\eta = \underline{w}^T\underline{x} = log(\frac{p(c=1|x;\eta)}{p(c=0|x;\eta)}) = log(\frac{\phi}{1-\phi})$.
\begin{align}
\begin{split}
& \therefore p(c|x;\eta) = \sigma(\eta)^c\ (1-exp(\eta))^{1-c} = \sigma(\underline{w}^T\underline{x})^c\ (1-exp(\underline{w}^T\underline{x}))^{1-c}
\end{split}
\end{align}
#### Sufficient Statistics importance 
Before delving further into the topic of sufficient statistics, we need to derive a formula. As was mentioned before, the function $g(\eta)$ is used for normalization, so, we can easily take the gradient of this formula w.r.t $\eta$. And this is done as follows:-
\begin{align}
\begin{split}
&\nabla_\eta (g(\eta)\int_{-\infty}^{\infty}h(x)exp(\eta^Tu(x))dx = 1);\ by\ using\ chain\ rule\\
& \Big(\nabla_\eta g(\eta)\Big)(\frac{g(\eta)}{g(\eta)}) \int_{-\infty}^{\infty}h(x)exp(\eta^Tu(x))dx +\ g(\eta)\int_{-\infty}^{\infty}h(x)exp(\eta^Tu(x)) u(x)dx =0\\
&By\ using\ g(\eta)\int_{-\infty}^{\infty}h(x)exp(\eta^Tu(x))dx = 1\ and\ E[g(x)]\ = \int_{}g(x)f_x(x)dx\\
&(\frac{-1}{g(\eta)} \nabla_\eta g(\eta)) = \int_{} u(x)\ h(x) g(\eta)\ exp(\eta^T u(x)) dx;\ by\ using\ \nabla_x ln(x) = \frac{1}{x}\\
&-\nabla_\eta ln(g(\eta)) = E[u(x)]\\
\end{split}
\end{align}
After showing the relationship between the sufficient statistics and the normalization term, we are now able to discuss the meaning of sufficient statistics. We now construct the negative log-likelihood(which is known as cross entropy) for the data and for brevity we will discuss the maximization of the likelihood with respect to the natural parameters.
\begin{align}
\begin{split}
&-\nabla_\eta ln(p(x|\eta))= -\Big( (\prod_{n=1}^{N}h(x_n))\ (g(\eta))^N\ exp(\eta^T \sum_{n=1}^{N}u(x_n))\Big) =0\\
&\nabla_\eta ln(g(\eta_{ML})) = \frac{1}{N} \sum_{n=1}^{N}u(x_n); as\ N \rightarrow \infty\\
\end{split}
\end{align}
As can be seen from the last formula, to maximize the likelihood w.r.t the natural parameters we need only the sufficient values of x, and this indicate that there is no need to store the value of x, hence, the name of sufficient statistics.

#### Mean value of the conditional distribution of the target class
\begin{align}
\begin{split}
E[u(c)|x;\eta] = \sum_{c=0}^{1}u(c)\ p(c|x;\eta) = p(c=1|x;\eta)=\sigma(-\eta)exp(\eta^T) = (1-\phi)\ \frac{\phi}{1-\phi}=\phi\\
= \frac{exp(\eta)}{1+exp(\eta)}=\ \sigma(\eta)\\
\end{split}
\end{align}
And this formula is known as the canonical response function, in which it connect the response("target") to our non-linear function w.r.t the parameter('w'). And this non-linear function is known as the sigmoid function.


In [ ]:
# Visualization for the sigmoidal function
#See how the sigmoidal is able to accurately differentiate between the binary classes
x = np.linspace(-10, 10, 1000)
y = np.zeros((len(x), 1))
y = [1 if t >= 0 else 0 for t in x]
colors = ['black|' if t == 0 else 'yellow' for t in y ]
sigmoid_fit = 1/(1 + np.exp(-1 * x))#Using broadcasting property of python
plt.scatter(x, y, c=y)
plt.plot(x, sigmoid_fit, c='red')
plt.grid('on')
plt.show()

# Loading Data, visualization and transformation

In [ ]:
data = pd.read_csv("/kaggle/input/breast-cancer-wisconsin-data/data.csv")
data = data.drop(["id", 'Unnamed: 32'], axis=1)
print(data.columns)
print(data.shape)
print(f"# columns: {len(data.columns)}")
data["diagnosis"] = data["diagnosis"].map({"M": 1, "B": 0})# benign(not deadly), malignant(deadly)
data.describe()


In [ ]:
#if we didn't print anything in the loop then there are no missing values.
print(f"# columns: {len(data.columns)}")
# Check for missing values
for column in data.columns:
    if data[column].isna().values.sum() > 0:
        print(f"Missing values at {column}")
        print(f"#Missing values is {data[column].isna().values.sum()}")

In [ ]:
# Splitting the data into training set and test set
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_data = None
test_data = None
for train_ind, test_ind in splitter.split(data, data['diagnosis']):
    train_data = data.iloc[train_ind]
    test_data = data.iloc[test_ind]

In [ ]:
fig, axes = plt.subplots(1, 2)
train_data["diagnosis"].hist(ax=axes[0], density=True)
axes[0].set_title("Distribution for the labels of the training set")
test_data["diagnosis"].hist(ax=axes[1], density=True)
axes[1].set_title("Distribution for the labels of the test set")
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=2, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
plt.show()

In [ ]:
#Normalizing the dataset since all columns of the dataset are numeric we can standard normalize it to be of mean 0 with unit variance
X_train = np.asarray(train_data.drop("diagnosis", axis=1))
y_train = np.asarray(train_data["diagnosis"])
X_test = np.asarray(test_data.drop("diagnosis", axis=1))
y_test = np.asarray(test_data["diagnosis"])

standarize = StandardScaler()
standarize.fit(X_train)
X_train = standarize.transform(X_train)
X_test = standarize.transform(X_test)
print(np.mean(X_train, axis=0))
print(np.std(X_train, axis=0))

# Maximizing the log-likelihood
After expressing the conditional distribution of the target variable, now, comes the part of maximizing the negative log likelihood( which is equivalent to the statement of minimizing to be 0). Each observation is assumed to be derived from i.i.d that is distributed according to the exponential representation of the Bernoulli distribution. The maximization of the cross entropy cost function would be with respect to $\underline{w}$. And for brevity I will not discuss the derivation in details, but I need to state that it can be easily done using chain rule from calculus.
\begin{align}
\begin{split}
&\nabla_{\underline{w}}\ell(\eta) = \frac{1}{m}\nabla_{\underline{w}}\ \Big(-log\Big( \prod_{n=1}^{N}p(c|\underline{x}; \eta)\Big)\Big)\\
&=\frac{1}{m}\nabla_{\underline{w}}\ \Big(-\sum_{n=1}^{N}c\ log{\phi}+ (1-c)\ log(1-\phi)\Big)\\
&=\frac{1}{m}\nabla_{\underline{w}}\ \Big(-\sum_{n=1}^{N}c\log(\sigma(\eta))+(1-c) log(1-\sigma(\eta))\Big); where\ is\ \eta = \underline{w}^T\underline{x}\\
&=-\frac{1}{m}\sum_{n=1}^{N}(c_n - \phi)\underline{x_n} =-\frac{1}{m}\sum_{n=1}^{N}(c_n - E[u(c)|\underline{x}; \eta])\underline{x_n}\\
&= \frac{1}{m}\sum_{n=1}^{N}(\phi - c_n)\underline{x_n}\\
&= \frac{1}{m}\sum_{n=1}^{N}(\sigma(\underline{w}^{(t)}\underline{x_n}) - c_n)\underline{x_n}
\end{split}
\end{align}

# Update Step
In which the point estimate of $\underline{w}$ can be found using gradient descent, in which I will utilize the following update method for w:- 

\begin{align}
\begin{split}
&let\ X\in R^{mxn}\ and\ w \in R^m \ c \in R^n\\
&\underline{w}^{(t+1)} = \underline{w}^{(t)} - \alpha \nabla_{\underline{w}^{(t)}}\ell(\eta) = \underline{w}^{(t)} -  \frac{\alpha}{m}\sum_{n=1}^{N}(\sigma(\underline{w}^{(t)}\underline{x_n}) - c_n)\underline{x_n}\\
&In\ matrix\ form -> \underline{w}^{(t+1)} = \underline{w}^{(t)} + \alpha*X^T(\underline{c} - \sigma(X \cdot w))
\end{split}
\end{align}

# Regularization
Also, its known that maximizing the likelihood tends to over fit to the data especially if the number of observations isn't large, so, to solve this we would add the L2-regularizing(Ridge regularization) term to the cost function. And this is equivalent to a maximum a posteriori (MAP) with the prior as the Gaussian distribution with mean zero and precision of $\lambda^-1$. Therefore, the cost function which is the binary cross entropy and optimization for $\underline{w}$ would be seen in the following equation.

\begin{align}
\begin{split}
&J(\underline{w}) = \frac{1}{m}(-\ell(\underline{w}) + \frac{\lambda}{2}\underline{w}^T\underline{w}) = \frac{-1}{m} \sum_{n=1}^{N} c\ log{\phi}+ (1-c)\ log(1-\phi) - \frac{\lambda}{2m}\underline{w}^T\underline{w}\\
&\underline{w}^{(t+1)} = \underline{w}^{(t)} -\alpha\Big(\frac{1}{m}\sum_{n=1}^{N}(\sigma(\underline{w}^T\underline{x_n}) - c)\underline{x_n}\ + \frac{\lambda}{2m}\underline{w}^{(t)}\Big)\\
&=\underline{w}^{(t)} - \alpha\Big(\frac{1}{m}X^T(\sigma(X\underline{w}) - \underline{y}) + \frac{\lambda}{m}\underline{w}\Big);\ X \in R^{m\ x\ n}\\
\end{split}
\end{align}


# Building Logistic Regression from scratch and training the model

In [ ]:
def sigmoid_function(X, w, n):
    assert(X.shape == (n, w.shape[0]))
    activation = np.divide(1.0, 1.0 + np.exp(-1.0 * X.dot(w)))
    return activation

#I willn't do regularization because it can be easily implemented
class LogisticRegression(object):
    
    def __init__(self, m):
        np.random.seed(42)
        self.w = np.random.rand(m + 1, 1)
        self.w[0] = 0 #bias term

    def fit(self, alpha, epochs, mini_batch_size, X, y):
        y = y.reshape(-1, 1)
        assert(y.shape[0] == X.shape[0])
        
        #Data is already shuffled
        m = self.w.shape[1]
        X_padded = np.ones((X.shape[0], X.shape[1] + 1))
        X_padded[:, 1:] = X.copy()
        n = X_padded.shape[0]

        number_batches = (int)(n/mini_batch_size)
        print(number_batches)
        costs = []
        for epoch in range(0, epochs):
            counter = 0
            cost = []
            for batch in range(0, number_batches):
                X_t = X_padded[counter:counter+mini_batch_size, :]
                y_t = y[counter:counter+mini_batch_size]
                self.w = self.w + alpha * X_t.T.dot(y_t - sigmoid_function(X_t, self.w, len(X_t)))
                cost.append(self.binaryCrossEntropy(y_t, X_t))
                counter += mini_batch_size
            if counter < len(X_padded):
                X_t = X_padded[counter:, :]
                y_t = y[counter:]
                self.w = self.w - alpha * X_t.T.dot(y_t - sigmoid_function(X_t, self.w, len(X_t)))
                cost.append(self.binaryCrossEntropy(y_t, X_t))
                
            costs.append(np.mean(cost))
            if (epoch % 2) == 0:
                print(f"Cost at {epoch} is: {costs[epoch]}")
        return costs
    
    def predict(self, X, decision_boundary):
        X_t = X
        if abs(X.shape[1] - self.w.shape[0]) == 1:
            #Pad it 
            X_t = np.ones((X.shape[0], X.shape[1] + 1))
            X_t[:, 1:] = X
        return (sigmoid_function(X_t, self.w, X_t.shape[0]) >= decision_boundary) * 1
    
    def binaryCrossEntropy(self, y_t, X_t):
        assert(y_t.shape[0] == X_t.shape[0])
        return np.mean(-1.0 * (y_t * np.log(sigmoid_function(X_t, self.w, X_t.shape[0]) + 0.0000001).reshape(-1, 1) + (1 - y_t) * np.log((1 - sigmoid_function(X_t, self.w, X_t.shape[0])) + 0.0000001).reshape(-1, 1)))




In [ ]:
model = LogisticRegression(X_train.shape[1])
costs = model.fit(0.8, 10, 64, X_train, y_train)
prediction = model.predict(X_train, 0.6)
cnf = confusion_matrix(y_train, prediction)
cnf

# Performance of the model on the test set

In [ ]:
prediction = model.predict(X_test, 0.6)
cnf = confusion_matrix(y_test, prediction)
print(f"Accuracy: {np.round(np.sum(np.diag(cnf))/np.sum(cnf), 4) * 100 }%")
cnf

# References 
* Chapter 1, chapter 2 and Chapter 4 from Bishop, C. (2006). Pattern Recognition and Machine Learning. Cambridge: Springer.
* Andrew Ng, Lec 1: (https://www.youtube.com/watch?v=UzxYlbK2c7E)
* Andrew Ng, Lec 2: (https://www.youtube.com/watch?v=5u4G23_OohI)
* Andrew Ng, Lec 3: (https://www.youtube.com/watch?v=HZ4cvaztQEs)
* Andrew Ng, Lec 4: (https://www.youtube.com/watch?v=nLKOQfKLUks)
